In [2]:
# importing the required libraries 
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Pre-Post Analysis Of Drug Related Deaths In Florida 

In [3]:
# loading pre-cleaned data on deaths in Florida and its neighbouring states 
deaths = pd.read_csv(r"C:\Users\annap\OneDrive\Desktop\Opioid Project\Deaths.csv")

In [4]:
deaths.head(3)

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State,StateFIPS,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,FIP_unique,Population,county_test
0,Autauga,1001.0,2007.0,2007.0,All other non-drug and non-alcohol causes,O9,399.0,AL,1,Autauga,Alabama,1001,AL,AL | AUTAUGA,AL,10011,52405.0,Autauga
1,Autauga,1001.0,2008.0,2008.0,All other non-drug and non-alcohol causes,O9,448.0,AL,1,Autauga,Alabama,1001,AL,AL | AUTAUGA,AL,10011,53277.0,Autauga
2,Autauga,1001.0,2009.0,2009.0,All other non-drug and non-alcohol causes,O9,408.0,AL,1,Autauga,Alabama,1001,AL,AL | AUTAUGA,AL,10011,54135.0,Autauga


In [5]:
deaths.columns

Index(['County', 'County Code', 'Year', 'Year Code',
       'Drug/Alcohol Induced Cause', 'Drug/Alcohol Induced Cause Code',
       'Deaths', 'State', 'StateFIPS', 'CountyName', 'StateName', 'CountyFIPS',
       'StateAbbr', 'STATE_COUNTY', 'state_abbrev', 'FIP_unique', 'Population',
       'county_test'],
      dtype='object')

In [6]:
# creating a copy of deaths dataset, to prevent SettingWithCopy warnings. The irrelevant columns are dropped.

deaths_copy = deaths.copy()
deaths_copy.drop(["County Code","Year Code","Drug/Alcohol Induced Cause Code","State", "CountyName", "StateAbbr","STATE_COUNTY","county_test"],axis=1, inplace = True)
deaths_copy.head(3)

,County,Year,Drug/Alcohol Induced Cause,Deaths,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population
0,Autauga,2007.0,All other non-drug and non-alcohol causes,399.0,1,Alabama,1001,AL,10011,52405.0
1,Autauga,2008.0,All other non-drug and non-alcohol causes,448.0,1,Alabama,1001,AL,10011,53277.0
2,Autauga,2009.0,All other non-drug and non-alcohol causes,408.0,1,Alabama,1001,AL,10011,54135.0


In [7]:
# converting some of the columns to more appropriate data type - integer data type
deaths_copy["Year"] = deaths_copy["Year"].astype("int64")
deaths_copy["Deaths"] = deaths_copy["Deaths"].astype("int64")
deaths_copy["Population"] = deaths_copy["Population"].astype("int64")

deaths_copy.head(3)


,County,Year,Drug/Alcohol Induced Cause,Deaths,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population
0,Autauga,2007,All other non-drug and non-alcohol causes,399,1,Alabama,1001,AL,10011,52405
1,Autauga,2008,All other non-drug and non-alcohol causes,448,1,Alabama,1001,AL,10011,53277
2,Autauga,2009,All other non-drug and non-alcohol causes,408,1,Alabama,1001,AL,10011,54135


In [8]:
deaths_copy["StateName"].unique()


array(['Alabama', 'Florida', 'Georgia', 'Mississippi', 'South Carolina',
       'Tennessee'], dtype=object)

In [9]:
deaths_copy["Drug/Alcohol Induced Cause"].unique()

array(['All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other alcohol-induced causes',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Suicide (X60-X64)',
       'Alcohol poisonings (overdose) (X45, X65, Y15)'], dtype=object)

In [10]:
# creating the dataset for Florida which has all deaths that are drug related 

florida_deaths = deaths_copy[
    (deaths_copy["StateName"] == "Florida")
    & (
        (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Unintentional (X40-X44)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Undetermined (Y10-Y14)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Suicide (X60-X64)"
        )
    )
]

In [11]:
florida_deaths.head(5)

,County,Year,Drug/Alcohol Induced Cause,Deaths,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population
618,Alachua,2007,Drug poisonings (overdose) Unintentional (X40-...,17,12,Florida,12001,FL,1200112,242685
623,Alachua,2009,Drug poisonings (overdose) Unintentional (X40-...,15,12,Florida,12001,FL,1200112,246657
626,Alachua,2010,Drug poisonings (overdose) Unintentional (X40-...,15,12,Florida,12001,FL,1200112,247614
629,Alachua,2011,Drug poisonings (overdose) Unintentional (X40-...,14,12,Florida,12001,FL,1200112,249834
632,Alachua,2012,Drug poisonings (overdose) Unintentional (X40-...,14,12,Florida,12001,FL,1200112,251520


In [12]:
florida_deaths_copy = florida_deaths.copy()

In [13]:
# calculating the drug related death rates in Florida per 100000 people 

florida_deaths_copy["overdose_per_100k"] = (florida_deaths_copy["Deaths"] / florida_deaths_copy["Population"]) * 100_000

florida_deaths_copy.head(3)

,County,Year,Drug/Alcohol Induced Cause,Deaths,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population,overdose_per_100k
618,Alachua,2007,Drug poisonings (overdose) Unintentional (X40-...,17,12,Florida,12001,FL,1200112,242685,7.004965
623,Alachua,2009,Drug poisonings (overdose) Unintentional (X40-...,15,12,Florida,12001,FL,1200112,246657,6.081319
626,Alachua,2010,Drug poisonings (overdose) Unintentional (X40-...,15,12,Florida,12001,FL,1200112,247614,6.057816


In [14]:
# calculating and displaying year and county wise results for drug related deaths in Florida

florida_result = florida_deaths_copy.groupby(["Year", "County"])["overdose_per_100k"].sum().reset_index()
florida_result.head(5)

,Year,County,overdose_per_100k
0,2007,Alachua,7.004965
1,2007,Bay,18.143881
2,2007,Brevard,19.269286
3,2007,Broward,15.225255
4,2007,Citrus,12.058961


In [15]:
# calculating and displaying yearwise results for drug related deaths in Florida 

florida_yearwise_results = florida_deaths_copy.groupby("Year")["overdose_per_100k"].mean()
florida_yearwise_results

Year
2007    10.589778
2008    11.389320
2009    10.090938
2010    11.976183
2011    11.423950
2012    10.435346
2013     9.046919
Name: overdose_per_100k, dtype: float64

## Pre-Post Analysis Of Drug Related Prescriptions In Florida 

In [16]:
# loading data for drug prescriptions in Florida and neighbouring states 
prescriptions = pd.read_csv(r"C:\Users\annap\OneDrive\Desktop\Opioid Project\Prescriptions.csv",low_memory = False)

In [17]:
prescriptions.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'ACTION_INDICATOR', 'ORDER_FORM_NO',
       'CORRECTION_NO', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name',
       'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name',
       'Revised_Company_Name', 'Reporter_family', 'dos_str', 'Year', 'Month',
       'StateFIPS', 'CountyName', 'StateName', 'CountyFIPS', 'StateAbbr',
       'STATE_COUNTY', 'state_abbrev', 'FIP_unique', 'State', 'Population',
       'county_test'],
      dtyp

In [18]:
prescriptions.head(3)

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,state_abbrev,FIP_unique,State,Population,county_test
0,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701.0,SUFFOLK,...,palm beach,Florida,12099,FL,FL | PALM BEACH,FL,1209912,FL,1286586.0,Palm Beach
1,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701.0,SUFFOLK,...,palm beach,Florida,12099,FL,FL | PALM BEACH,FL,1209912,FL,1286586.0,Palm Beach
2,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701.0,SUFFOLK,...,palm beach,Florida,12099,FL,FL | PALM BEACH,FL,1209912,FL,1286586.0,Palm Beach


In [19]:
# reducing the number of columns in the drug prescriptions dataset by including only the relevant attributes to create a new dataset

prescriptions_reduced = prescriptions[['DRUG_CODE',
       'DRUG_NAME', 'QUANTITY', 'Product_Name',
       'Measure', 'Year', 'Month',
       'StateFIPS', 'StateName', 'CountyFIPS',
       'state_abbrev', 'FIP_unique', 'Population','county_test']]

In [20]:
prescriptions_reduced.head(5)

,DRUG_CODE,DRUG_NAME,QUANTITY,Product_Name,Measure,Year,Month,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population,county_test
0,9143.0,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007.0,2.0,12,Florida,12099,FL,1209912,1286586.0,Palm Beach
1,9143.0,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007.0,3.0,12,Florida,12099,FL,1209912,1286586.0,Palm Beach
2,9143.0,OXYCODONE,6.0,ENDOCET - 10MG OXYCODONE.HCL/325MG A,TAB,2007.0,4.0,12,Florida,12099,FL,1209912,1286586.0,Palm Beach
3,9143.0,OXYCODONE,12.0,OXYCODONE HYDROCHLORIDE USP 30MG TAB,TAB,2007.0,4.0,12,Florida,12099,FL,1209912,1286586.0,Palm Beach
4,9193.0,HYDROCODONE,1.0,"HYDROCOD.BIT.& APAP,10MG/660MG/TAB",TAB,2007.0,4.0,12,Florida,12099,FL,1209912,1286586.0,Palm Beach


In [21]:
# creating a copy of reduced dataset of prescriptions and converting some of the attributes to appropriate data type

prescriptions_reduced_copy = prescriptions_reduced.copy()

prescriptions_reduced_copy["Year"] = prescriptions_reduced_copy["Year"].astype("int64")
prescriptions_reduced_copy["DRUG_CODE"] = prescriptions_reduced_copy["DRUG_CODE"].astype("int64")
prescriptions_reduced_copy["Month"] = prescriptions_reduced_copy["Month"].astype("int64")
prescriptions_reduced_copy["Population"] = prescriptions_reduced_copy["Population"].astype("int64")

In [22]:
prescriptions_reduced_copy.head(3)

,DRUG_CODE,DRUG_NAME,QUANTITY,Product_Name,Measure,Year,Month,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population,county_test
0,9143,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007,2,12,Florida,12099,FL,1209912,1286586,Palm Beach
1,9143,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007,3,12,Florida,12099,FL,1209912,1286586,Palm Beach
2,9143,OXYCODONE,6.0,ENDOCET - 10MG OXYCODONE.HCL/325MG A,TAB,2007,4,12,Florida,12099,FL,1209912,1286586,Palm Beach


In [23]:
# creating a dataset that has all the drug prescriptions in the state of Florida 

florida_prescriptions = prescriptions_reduced_copy[prescriptions_reduced_copy["StateName"]=="Florida"]

florida_prescriptions.head()

,DRUG_CODE,DRUG_NAME,QUANTITY,Product_Name,Measure,Year,Month,StateFIPS,StateName,CountyFIPS,state_abbrev,FIP_unique,Population,county_test
0,9143,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007,2,12,Florida,12099,FL,1209912,1286586,Palm Beach
1,9143,OXYCODONE,3.0,OXYCODONE HCL 80MG TABS,TAB,2007,3,12,Florida,12099,FL,1209912,1286586,Palm Beach
2,9143,OXYCODONE,6.0,ENDOCET - 10MG OXYCODONE.HCL/325MG A,TAB,2007,4,12,Florida,12099,FL,1209912,1286586,Palm Beach
3,9143,OXYCODONE,12.0,OXYCODONE HYDROCHLORIDE USP 30MG TAB,TAB,2007,4,12,Florida,12099,FL,1209912,1286586,Palm Beach
4,9193,HYDROCODONE,1.0,"HYDROCOD.BIT.& APAP,10MG/660MG/TAB",TAB,2007,4,12,Florida,12099,FL,1209912,1286586,Palm Beach


In [24]:
# calculating and displaying total number of drug prescriptions that took place in Florida. results are grouped and displayed year and county wise 

florida_prescriptions_result = florida_prescriptions.groupby(["Year", "county_test"])["QUANTITY"].sum().reset_index()
florida_prescriptions_result.head(5)

,Year,county_test,QUANTITY
0,2007,Alachua,181.0
1,2007,Baker,3.0
2,2007,Bay,49.0
3,2007,Bradford,136.0
4,2007,Brevard,258.0


## Difference In Difference Analysis Of Drug Related Deaths In Florida And Other States 

In [25]:
# creating a new dataset, that contains drug related deaths in other neighboring states of Florida - Alabama, Georgia, Mississippi, South Carolin and Tennessee

other_states_deaths = deaths_copy[
    (deaths_copy["StateName"] != "Florida")
    & (
        (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Unintentional (X40-X44)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Undetermined (Y10-Y14)"
        )
        | (
            deaths_copy["Drug/Alcohol Induced Cause"]
            == "Drug poisonings (overdose) Suicide (X60-X64)"
        )
    )
]

In [26]:
other_states_deaths_copy = other_states_deaths.copy()

other_states_deaths_copy["overdose_per_100k"] = (other_states_deaths_copy["Deaths"] / other_states_deaths_copy["Population"]) * 100_000


In [27]:
# calculating and displaying results for drug related deaths - year, state and county wise 

other_states_result = other_states_deaths_copy.groupby(["Year", "StateName", "County"])["overdose_per_100k"].sum().reset_index()
other_states_result.head()


,Year,StateName,County,overdose_per_100k
0,2007,Alabama,Baldwin,13.920791
1,2007,Alabama,Calhoun,9.465541
2,2007,Alabama,Houston,10.232587
3,2007,Alabama,Jefferson,10.226463
4,2007,Alabama,Lauderdale,17.561768


In [28]:
# calculating yearwise drug related deaths results for Alabama 
alabama_results = other_states_deaths_copy[other_states_deaths_copy["StateName"] == "Alabama"].groupby("Year")["overdose_per_100k"].mean()
alabama_results

Year
2007    13.368704
2008    16.361755
2009    15.272194
2010    14.556800
2011    14.767924
2012    14.834840
2013    18.018408
Name: overdose_per_100k, dtype: float64

In [29]:
# calculating yearwise drug related deaths results for Georgia
georgia_results = other_states_deaths_copy[other_states_deaths_copy["StateName"] == "Georgia"].groupby("Year")["overdose_per_100k"].mean()
georgia_results

Year
2007    11.960843
2008    10.328943
2009    11.745507
2010    13.869420
2011    12.381299
2012    11.653082
2013    11.902365
Name: overdose_per_100k, dtype: float64

In [30]:
# calculating yearwise drug related deaths results for Mississippi
mississippi_results = other_states_deaths_copy[other_states_deaths_copy["StateName"] == "Mississippi"].groupby("Year")["overdose_per_100k"].mean()
mississippi_results

Year
2007    21.903231
2008    17.037801
2009    20.989875
2010    18.498209
2011    16.426466
2012    15.167236
2013    19.345719
Name: overdose_per_100k, dtype: float64

In [31]:
# calculating yearwise drug related deaths results for South Carolina
south_carolina_results = other_states_deaths_copy[other_states_deaths_copy["StateName"] == "South Carolina"].groupby("Year")["overdose_per_100k"].mean()
south_carolina_results

Year
2007    12.447756
2008    12.431124
2009    13.862295
2010    14.429518
2011    12.253051
2012    12.683164
2013    13.080714
Name: overdose_per_100k, dtype: float64

In [32]:
# calculating yearwise drug related deaths results for Tennessee
tennessee_results = other_states_deaths_copy[other_states_deaths_copy["StateName"] == "Tennessee"].groupby("Year")["overdose_per_100k"].mean()
tennessee_results

Year
2007    16.285685
2008    14.739367
2009    14.611214
2010    17.048880
2011    18.678175
2012    16.506248
2013    20.844220
Name: overdose_per_100k, dtype: float64